In [ ]:
#Standard Imports für alle Notebooks

!pip3 install tira ir-datasets python-terrier

from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt

ensure_pyterrier_is_loaded()
tira = Client()

pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

In [ ]:
#Erstellen von costum Retrieval Systemen

def create_index(documents, stopwords):
    # Stemmer has been added 
    indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer='EnglishSnowballStemmer', stopwords=stopwords)
    # indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer='PorterStemmer', stopwords=stopwords)
    # indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer='WeakPorterStemmer', stopwords=stopwords)
    # # indexer = pt.IterDictIndexer("/tmp/index", overwrite=True, meta={'docno': 100, 'text': 20480}, stemmer='StemmerTermPipeline', stopwords=stopwords)
    
    index_ref = indexer.index(documents)
    return pt.IndexFactory.of(index_ref)

def read_text_file_to_array(file_path):
    try:
        with open(file_path, 'r') as file:
            lines = file.readlines()
            # Remove newline characters and convert to integers
            array = [(line.strip()) for line in lines]
            return array
    except FileNotFoundError:
        print(f"File {file_path} not found.")
        return None

# Read txt file and convert it to an array
file_path = "../terrier-stopwordslist.txt"

stopwords = read_text_file_to_array(file_path)


costum_index = create_index(pt_dataset.get_corpus_iter(), stopwords)

XSqrA_M	 = pt.BatchRetrieve(index, wmodel="XSqrA_M")
XSqrA_M_costum	 = pt.BatchRetrieve(costum_index, wmodel="XSqrA_M")

In [ ]:
#Evaluation

pt.Experiment(
    [XSqrA_M, XSqrA_M_costum],
    pt_dataset.get_topics(),
    pt_dataset.get_qrels(),
    ["ndcg_cut.10", "recip_rank", "recall_100", "P_10"],
    names = ["XSqrA_M", "XSqrA_M_costum"]
)